In [1]:
from sklearn import metrics, random_projection, cross_validation, datasets, grid_search, linear_model, metrics, random_projection, manifold, preprocessing
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from collections import  Counter
import numpy as np
import pandas as pd
from sklearn.decomposition import RandomizedPCA
from sklearn import manifold
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.svm import SVC, OneClassSVM
from sklearn.decomposition import TruncatedSVD
import xgboost as xgb
from scipy import load, save
import sklearn
%pylab inline

/home/tosha/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/tosha/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Populating the interactive namespace from numpy and matplotlib


In [2]:
dataframe = pd.read_csv('dataframe/genres_40.csv')
dataframe.loc[:, 'm1':] = preprocessing.normalize(dataframe.loc[:, 'm1':])

labels = ['hiphop', 'classical', 'jazz', 'rock', 'metal', 'synth', 'pop', 'blues']
# labels_n = ['neoclassic', 'jazz',  'metal', 'pop']

# dataframe = dataframe[dataframe['tag'] == 'neoclassic'].append(dataframe[dataframe['tag'] == 'jazz']).append(dataframe[dataframe['tag'] == 'pop']).append(dataframe[dataframe['tag'] == 'metal'])
# dataframe.append(dataframe[dataframe['tag'] == 'pop'])
# dataframe = dataframe.append(dataframe[dataframe['tag'] == 'jazz'])
# dataframe = dataframe.append(dataframe[dataframe['tag'] == 'metal'])
dataframe = dataframe.sample(frac=1)

shape(dataframe)

(1601, 201)

In [3]:
def search_abnom(dataframe, label, g, nu):
    svm_ = OneClassSVM(gamma=g, nu=nu)
    svm_.fit(dataframe[dataframe['tag'] == label].loc[:, 'm1':])
    l = svm_.predict(dataframe[dataframe['tag'] == label].loc[:, 'm1':])
    return l

labels = ['hiphop', 'classical', 'jazz', 'rock', 'metal', 'synth', 'pop', 'blues']
for label in labels:
    svm_label = search_abnom(dataframe, label, 2, 0.15).tolist()
    dataframe[dataframe['tag'] == label] = dataframe[dataframe['tag'] == label][np.array(svm_label) == 1]
    dataframe = dataframe.dropna()
print(shape(dataframe))

(1349, 201)


In [4]:
# svm_label = []
# for label in labels:
#     svm_label += search_abnom(label, 2, 0.001).tolist()
# dataframe_svm = dataframe[np.array(svm_label) == 1]

# dataframe_svm_t = np.array(dataframe_svm.loc[:, 'm1':])
# dataframe_svm_l = np.array(dataframe_svm['tag'])

dataframe_svm_t = np.array(dataframe.loc[:, 'm1':])
dataframe_svm_l = np.array(dataframe['tag'])
np.shape(dataframe_svm_l)

(1349,)

In [5]:
kmeans1 = KMeans(n_clusters=4, random_state=10)
preds1 = kmeans1.fit_predict(dataframe_svm_t)
kmeans2 = KMeans(n_clusters=8, random_state=10)
preds2 = kmeans2.fit_predict(dataframe_svm_t)
kmeans3 = KMeans(n_clusters=12, random_state=10)
preds3 = kmeans3.fit_predict(dataframe_svm_t)

In [6]:
svcc = SVC(C=10, gamma=5, kernel = 'rbf', probability=True)
lrc = LogisticRegression(C = 430)
knc = KNeighborsClassifier(weights = 'distance', n_neighbors=7, metric = 'manhattan')
rfc = RandomForestClassifier(max_depth=23, n_estimators=1300, random_state=1)
xgbc = xgb.XGBClassifier(silent=False, nthread=4, max_depth=2, n_estimators=4000, learning_rate=0.03)

In [38]:
def fin_classif(res_model, list_param, name_file):
    res_list = []
    list_preds_f = []
    list_labels_f = []
    for train_ind, test_ind in cross_validation.StratifiedKFold(dataframe_svm_l, n_folds=5, random_state=1):
        np.random.shuffle(train_ind) 
        np.random.shuffle(test_ind)
        
        df_res = dataframe_svm_t[:, list_param>0]
        df_res = np.hstack((df_res, preds1.reshape(len(preds1), 1)/4))
        df_res = np.hstack((df_res, preds2.reshape(len(preds2), 1)/8))
        df_res = np.hstack((df_res, preds3.reshape(len(preds3), 1)/12))
              
        df_b = dataframe_svm_t[:, load('boost_param.npy')]
#         rfc.fit(df_b[train_ind], dataframe_svm_l[train_ind])
#         boost_pred = rfc.predict_proba(df_b)
#         df_res = np.hstack((df_res, boost_pred))

        df_regr = dataframe_svm_t[:, load('regr_param.npy')]
        svcc.fit(df_regr[train_ind], dataframe_svm_l[train_ind])
        svcc_pred = svcc.predict_proba(df_regr)
        df_res = np.hstack((df_res, svcc_pred))

#         lrc.fit(df_regr[train_ind], dataframe_svm_l[train_ind])
#         lrc_pred = lrc.predict_proba(df_regr)
#         df_res = np.hstack((df_res, lrc_pred))

#         knc.fit(dataframe_svm_t[train_ind], dataframe_svm_l[train_ind])
#         knc_pred = knc.predict_proba(dataframe_svm_t)
#         df_res = np.hstack((df_res, knc_pred))
        
        res_model.fit(df_res[train_ind], dataframe_svm_l[train_ind])
        list_preds = res_model.predict(df_res[test_ind])
        res = metrics.accuracy_score(dataframe_svm_l[test_ind], list_preds)
        print(res)
        res_list.append(res)
        
        if len(list_preds_f) == 0:
            list_preds_f = list_preds
            list_labels_f = dataframe_svm_l[test_ind]
        else:
            list_preds_f = np.vstack((list_preds_f, list_preds))
            list_labels_f = np.hstack((list_labels_f, dataframe_svm_l[test_ind]))
    
    save(name_file, list_preds_f)
    save('labels_m.npy', list_labels_f        
    return res_list

SyntaxError: invalid syntax (<ipython-input-38-09cf5f14ba55>, line 47)

In [27]:
rl = fin_classif(rfc, load('boost_param.npy'))
print('mean =', np.mean(rl))
print('std = ', np.std(rl))

0.8791208791208791
0.8529411764705882
0.8592592592592593
0.9063670411985019
0.8764044943820225
mean = 0.8748185700862502
std =  0.018638903150386684


In [26]:
rl = fin_classif(xgbc, load('boost_param.npy'))
print('mean =', np.mean(rl))
print('std = ', np.std(rl))

/home/tosha/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.8754578754578755


/home/tosha/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.8566176470588235


/home/tosha/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.8407407407407408


/home/tosha/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.8913857677902621
0.8614232209737828
mean = 0.8651250504042969
std =  0.01718759181384516


/home/tosha/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [25]:
rl = fin_classif(lrc, load('regr_param.npy'))
print('mean =', np.mean(rl))
print('std = ', np.std(rl))

0.8974358974358975
0.8713235294117647
0.8777777777777778
0.898876404494382
0.8689138576779026
mean = 0.882865493359545
std =  0.01282494343162518


In [40]:
rl = fin_classif(knc, load('boost_param.npy'))
print('mean =', np.mean(rl))
print('std = ', np.std(rl))

0.9010989010989011
0.8602941176470589
0.8592592592592593
0.8838951310861424
0.8689138576779026
mean = 0.8746922533538528
std =  0.015880034501407035


In [16]:
rl = fin_classif(svcc, load('regr_param.npy'))
print('mean =', np.mean(rl))
print('std = ', np.std(rl))

0.8717948717948718
0.8644688644688645
0.8302583025830258
0.8364312267657993
0.846441947565543
mean = 0.8498790426356209
std =  0.015942379265744204
